# Toy modelling demonstration

This notebook demonstrates the use of several other scripts contained in this directory:
 - **[toy_data_generation.py](./toy_data_generation.py)**: generates "toy" datasets using known, specified generative processes
 - **[param_dict_generation.py](./param_dict_generation.py)**: generates parameter dicts specifying candidate models to train on the toy (or other, real) datasets
 - **[model_wrappers.py](./model_wrappers.py)**: presents a common interface for scikit-learn, statsmodels and XGBoost models, allowing multiple model types to be specified using the parameter dicts generated by param_dict_generation.py
 
The output is a csv file evaluating the performance of several different models on several toy datasets.

# Imports and settings

In [1]:
import multiprocessing
import pandas as pd
from sklearn.model_selection import train_test_split

from model_wrappers import try_models
from toy_data_generation import generate_linear_data, generate_interaction_data
from param_dict_generation import generate_ols_list, generate_mlp_list, generate_xgb_list

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)

# Dataset generation

In [3]:
datasets = []
for nrows in [1000, 3000]:
    for nvars in [10, 30]:
        datasets.append((*generate_linear_data(nrows, nvars), f'linear_{nvars}vars_{nrows}rows'))
        datasets.append((*generate_interaction_data(nrows, nvars),
                         f'interaction_{nvars}vars_{nrows}rows'))
        print(f'Datasets with {nrows} rows and {nvars} vars successfully appended')

Datasets with 1000 rows and 10 vars successfully appended
Datasets with 1000 rows and 30 vars successfully appended
Datasets with 3000 rows and 10 vars successfully appended
Datasets with 3000 rows and 30 vars successfully appended


# Experiment list generation

## Statsmodels list

In [4]:
ols_list = generate_ols_list(alpha_list=[0.01, 1], L1_wt_list=[0, 0.5, 1])

statsmodels_param_list = ols_list

## Scikit list

In [5]:
n_cpus = multiprocessing.cpu_count()

In [6]:
mlp_list = generate_mlp_list(hidden_layout_list=[(200,), (50, 50)])

xgb_list = generate_xgb_list(depth_list = [3,5,7],
                             learning_rate_list = [0.03, 0.1, 0.3],
                             n_jobs=max(1, n_cpus-1),
                             objective='Linear',
                             early_stopping_rounds=10)

scikit_param_list = mlp_list

xgb_param_list = xgb_list

# Model training and evaluation

In [7]:
results = {}

for X, y, dataset_name in datasets:
    print(f'\n\n{dataset_name}\n')
    # update scikit_param_list for xgboost early stopping eval_sets if present
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=303)
    
    scikit_results = try_models(
            X_train, X_test, y_train, y_test,
            param_list=scikit_param_list, model='scikit'
        )
    for key in scikit_results:
        results[(dataset_name, key)] = scikit_results[key]
        
    xgb_results = try_models(
            X_train, X_test, y_train, y_test,
            param_list=xgb_param_list, model='xgb'
        )
    for key in xgb_results:
        results[(dataset_name, key)] = xgb_results[key]
        
    sm_results = try_models(
            X_train, X_test, y_train, y_test,
            param_list=statsmodels_param_list, model='statsmodels'
        )
    for key in sm_results:
        results[(dataset_name, key)] = sm_results[key]
    
display(pd.DataFrame(results).T)



linear_10vars_1000rows

MLP_200 trained in 0.84 seconds
MLP_50_50 trained in 0.46 seconds
XGB_Linear_maxd_3_lr_0.03 trained in 0.36 seconds
XGB_Linear_maxd_3_lr_0.1 trained in 0.08 seconds
XGB_Linear_maxd_3_lr_0.3 trained in 0.03 seconds
XGB_Linear_maxd_5_lr_0.03 trained in 0.26 seconds
XGB_Linear_maxd_5_lr_0.1 trained in 0.07 seconds
XGB_Linear_maxd_5_lr_0.3 trained in 0.03 seconds
XGB_Linear_maxd_7_lr_0.03 trained in 0.42 seconds
XGB_Linear_maxd_7_lr_0.1 trained in 0.15 seconds
XGB_Linear_maxd_7_lr_0.3 trained in 0.05 seconds
OLS_no_reg trained in 0.01 seconds
OLS_alpha_0.01_L1_0 trained in 0.01 seconds
OLS_alpha_0.01_L1_0.5 trained in 0.09 seconds
OLS_alpha_0.01_L1_1 trained in 0.08 seconds
OLS_alpha_1_L1_0 trained in 0.01 seconds
OLS_alpha_1_L1_0.5 trained in 0.08 seconds
OLS_alpha_1_L1_1 trained in 0.01 seconds


interaction_10vars_1000rows

MLP_200 trained in 6.05 seconds
MLP_50_50 trained in 3.05 seconds
XGB_Linear_maxd_3_lr_0.03 trained in 0.22 seconds
XGB_Linear_maxd_3_lr_0.

/home/arthurjackrussell/repos/customer-deliverables/bm-venv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLP_200 trained in 7.62 seconds
MLP_50_50 trained in 4.35 seconds
XGB_Linear_maxd_3_lr_0.03 trained in 0.27 seconds
XGB_Linear_maxd_3_lr_0.1 trained in 0.06 seconds
XGB_Linear_maxd_3_lr_0.3 trained in 0.04 seconds
XGB_Linear_maxd_5_lr_0.03 trained in 0.27 seconds
XGB_Linear_maxd_5_lr_0.1 trained in 0.18 seconds
XGB_Linear_maxd_5_lr_0.3 trained in 0.05 seconds
XGB_Linear_maxd_7_lr_0.03 trained in 0.74 seconds
XGB_Linear_maxd_7_lr_0.1 trained in 0.23 seconds
XGB_Linear_maxd_7_lr_0.3 trained in 0.08 seconds
OLS_no_reg trained in 0.02 seconds
OLS_alpha_0.01_L1_0 trained in 0.02 seconds
OLS_alpha_0.01_L1_0.5 trained in 0.12 seconds
OLS_alpha_0.01_L1_1 trained in 0.12 seconds
OLS_alpha_1_L1_0 trained in 0.02 seconds
OLS_alpha_1_L1_0.5 trained in 0.07 seconds
OLS_alpha_1_L1_1 trained in 0.05 seconds


linear_10vars_3000rows

MLP_200 trained in 0.54 seconds
MLP_50_50 trained in 0.34 seconds
XGB_Linear_maxd_3_lr_0.03 trained in 0.58 seconds
XGB_Linear_maxd_3_lr_0.1 trained in 0.19 seconds
XGB_L

mse_test  \
linear_10vars_1000rows      MLP_200                      0.102773   
                            MLP_50_50                    0.103431   
                            XGB_Linear_maxd_3_lr_0.03    0.159965   
                            XGB_Linear_maxd_3_lr_0.1     0.157796   
                            XGB_Linear_maxd_3_lr_0.3     0.169041   
                            XGB_Linear_maxd_5_lr_0.03    0.191996   
                            XGB_Linear_maxd_5_lr_0.1     0.193676   
                            XGB_Linear_maxd_5_lr_0.3     0.242337   
                            XGB_Linear_maxd_7_lr_0.03    0.220254   
                            XGB_Linear_maxd_7_lr_0.1     0.229660   
                            XGB_Linear_maxd_7_lr_0.3     0.267698   
                            OLS_no_reg                   0.101629   
                            OLS_alpha_0.01_L1_0          0.125488   
                            OLS_alpha_0.01_L1_0.5        0.175789   
                            OLS_alpha_0.01_L1_1          0.218399   
                            OLS_alpha_1_L1_0             1.190380   
                            OLS_alpha_1_L1_0.5           1.746140   
                            OLS_alpha_1_L1_1             1.582732   
interaction_10vars_1000rows MLP_200                      2.661896   
                            MLP_50_50                    3.508805   
                            XGB_Linear_maxd_3_lr_0.03   17.051209   
                            XGB_Linear_maxd_3_lr_0.1    17.134733   
                            XGB_Linear_maxd_3_lr_0.3    15.069409   
                            XGB_Linear_maxd_5_lr_0.03   12.135053   
                            XGB_Linear_maxd_5_lr_0.1    11.206981   
                            XGB_Linear_maxd_5_lr_0.3    11.434322   
                            XGB_Linear_maxd_7_lr_0.03   12.237996   
                            XGB_Linear_maxd_7_lr_0.1    12.453545   
                            XGB_Linear_maxd_7_lr_0.3    12.580397   
                            OLS_no_reg                  23.764966   
                            OLS_alpha_0.01_L1_0         23.752810   
                            OLS_alpha_0.01_L1_0.5       23.759239   
                            OLS_alpha_0.01_L1_1         23.768107   
                            OLS_alpha_1_L1_0            29.581349   
                            OLS_alpha_1_L1_0.5          28.589312   
                            OLS_alpha_1_L1_1            27.100400   
linear_30vars_1000rows      MLP_200                      3.503491   
                            MLP_50_50                    3.030843   
                            XGB_Linear_maxd_3_lr_0.03    0.631897   
                            XGB_Linear_maxd_3_lr_0.1     0.592074   
                            XGB_Linear_maxd_3_lr_0.3     0.671505   
                            XGB_Linear_maxd_5_lr_0.03    0.755383   
                            XGB_Linear_maxd_5_lr_0.1     0.787626   
                            XGB_Linear_maxd_5_lr_0.3     1.012054   
                            XGB_Linear_maxd_7_lr_0.03    0.912056   
                            XGB_Linear_maxd_7_lr_0.1     0.949542   
                            XGB_Linear_maxd_7_lr_0.3     1.107405   
                            OLS_no_reg                   0.280421   
                            OLS_alpha_0.01_L1_0          0.374160   
                            OLS_alpha_0.01_L1_0.5        0.527640   
                            OLS_alpha_0.01_L1_1          0.606919   
                            OLS_alpha_1_L1_0             3.204950   
                            OLS_alpha_1_L1_0.5           3.908945   
                            OLS_alpha_1_L1_1             3.828709   
interaction_30vars_1000rows MLP_200                    113.160227   
                            MLP_50_50                  211.729613   
                            XGB_Linear_maxd_3_lr_0.03  248.042705   
                            XGB_Linear_maxd_3_lr_0.1   

# Output results

In [8]:
pd.DataFrame(results).T.to_csv('results.csv')